In [1]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.nn import Linear,Softmax,CrossEntropyLoss,ReLU,Sequential, Module
from torch.optim import Adam, SGD
import torch.nn.functional as F
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
import os
from PIL import Image
from time import time
from torchvision.models import resnet18, alexnet, vgg11, efficientnet_b0, resnet152
from tqdm import tqdm_notebook
device = 'cuda'

In [2]:
label_dict = dict([(j,i) for (i,j) in sorted(list(enumerate(os.listdir('../Dataset/CIFAR-10-images/train/'))))])
print(label_dict)

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


In [3]:
paths = []
label = []
for i in os.listdir('../Dataset/CIFAR-10-images/train/'):
    for j in os.listdir('../Dataset/CIFAR-10-images/train/'+str(i))[:500]:
        paths.append('../Dataset/CIFAR-10-images/train/'+str(i)+'/'+str(j))
        label.append(i)
train_df = pd.DataFrame({"paths": paths, "label": label})  

paths = []
label = []
for i in os.listdir('../Dataset/CIFAR-10-images/test/'):
    for j in os.listdir('../Dataset/CIFAR-10-images/test/'+str(i))[:500]:
        paths.append('../Dataset/CIFAR-10-images/test/'+str(i)+'/'+str(j))
        label.append(i)
test_df = pd.DataFrame({"paths": paths, "label": label})

In [4]:
print("train classes: ", len(train_df['label'].unique()))
print("train images: ", len(train_df['paths'].unique()))
print("test classes: ", len(test_df['label'].unique()))
print("test images: ", len(test_df['paths'].unique()))

train classes:  10
train images:  5000
test classes:  10
test images:  5000


In [5]:
class CIFARDataSet(Dataset):
    def __init__(self, csv_file, transform=None):
        self.transform = transform
        self.df = csv_file
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        label = label_dict[self.df.iloc[idx]['label']]
        label_onehot = np.zeros(10)
        label_onehot[label] = 1.
        path = self.df.iloc[idx]['paths']
        image = Image.open(path)
        image = self.transform(image)
        sample = {
            "image": image.to(device),
            "label": torch.from_numpy(label_onehot).to(device)
        }
        return sample

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
#         x = self.softmax(x)
        return x

In [7]:
model = resnet18()
model.fc = nn.Linear(in_features=512, out_features=10, bias=True)
model = model.to(device)

In [8]:
train_batch_size = 32
num_of_epochs = 20
learning_rate=0.01

In [9]:
traindataset = CIFARDataSet(train_df, transform=transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
]))
trainloader = DataLoader(traindataset, batch_size=train_batch_size,shuffle=True)
valdataset = CIFARDataSet(test_df, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
]))
valloader = DataLoader(valdataset, batch_size=32)

In [10]:
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.bn2.weight
	 layer

In [11]:
criterian = CrossEntropyLoss().to(device)
optimizer = Adam(params_to_update, lr=learning_rate)

In [12]:
train_f1 = []
test_f1 = []
print("----------------starting train loop------------------")
for epoch in range(num_of_epochs):
    model.train()
    start_time = time()
    print("epoch:",epoch)
    y_pred = []
    y_true = []
    total_loss = 0.0
    for batch in tqdm_notebook(trainloader):
        optimizer.zero_grad()
        output = model(batch['image'].to(device))
        loss = criterian(output,batch['label'])
        total_loss += loss
        loss.backward()
        optimizer.step()
        y_pred += list(output.argmax(dim=1).detach().cpu().numpy())
        y_true += list(batch['label'].argmax(dim=1).detach().cpu().numpy()) 
    print("training acc:",accuracy_score(y_pred,y_true),end=' ')
    f1 = f1_score(y_pred,y_true, average='micro')
    train_f1.append(f1)
    print("total loss:", total_loss, end=' ')
    print("training f1_score:", f1)
    
    y_pred = []
    y_true = []
    model.eval()
    for batch in valloader:
        output = model(batch['image'].to(device))
        y_pred += list(output.argmax(dim=1).detach().cpu().numpy())
        y_true += list(batch['label'].argmax(dim=1).detach().cpu().numpy())
    print("test acc:",accuracy_score(y_pred,y_true),end=' ')
    f1 = f1_score(y_pred,y_true, average='micro')
    test_f1.append(f1)
    print("test f1_score:", f1)

----------------starting train loop------------------
epoch: 0


C:\Users\nitin\AppData\Local\Temp/ipykernel_35588/2026690136.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(trainloader):


  0%|          | 0/157 [00:00<?, ?it/s]

training acc: 0.1748 total loss: tensor(404.4487, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) training f1_score: 0.1748
test acc: 0.1974 test f1_score: 0.1974
epoch: 1


C:\Users\nitin\AppData\Local\Temp/ipykernel_35588/2026690136.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(trainloader):


...

  0%|          | 0/157 [00:00<?, ?it/s]

training acc: 0.524 total loss: tensor(209.9673, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) training f1_score: 0.524
test acc: 0.5488 test f1_score: 0.5488
epoch: 19


C:\Users\nitin\AppData\Local\Temp/ipykernel_35588/2026690136.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(trainloader):


  0%|          | 0/157 [00:00<?, ?it/s]

training acc: 0.5396 total loss: tensor(200.7547, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) training f1_score: 0.5396
test acc: 0.5538 test f1_score: 0.5538


In [19]:
total_loss.item()

200.7547052700611

In [32]:
print("Training acc:",23,end=' ')
print("| loss:", total_loss.item(), end=' ')
print("| F1:", f1, end=' ')
print("||", end=' ')

Training acc: 23 | loss: 200.7547052700611 | F1: 0.5538 || 

In [33]:
a = [(1,2), (3,4)]

In [39]:
a[1][0] = 1

TypeError: 'tuple' object does not support item assignment

In [40]:
a = []
a.append([1,2])

In [42]:
def make_change(x):
    x=10

In [43]:
a = 20
make_change(a)

In [47]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    processes = []
    for i in range(10):
        p = Process(target=f, args=(str(i),))
        p.start()
    processes.append(p)
    for p in processes:
        p.join()

main line
module name: __main__
parent process: 23120
process id: 35588


In [48]:
from torch.multiprocessing import Manager

In [49]:
from multiprocessing import Process, Pipe

In [50]:
parent_conn, child_conn = Pipe()

In [ ]:
child_conn.